# 使用GPU
默认使用cpu


In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x1a1ca8daa60>)

In [3]:
torch.cuda.device_count() # 查看可用GPU数量

1

定义两个函数方便后续使用

In [4]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu()) # 尝试存在GPU上
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [11]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.0933, 0.6926, 0.8433],
        [0.7057, 0.3788, 0.2171]], device='cuda:0')

计算X+Y需要决定在哪个GPU上进行操作


In [9]:
Z = X.cuda(0) # 多GPU可以指定别的GPU
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [12]:
Y+Z # Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! 不在同一GPU的报错

tensor([[1.0933, 1.6926, 1.8433],
        [1.7057, 1.3788, 1.2171]], device='cuda:0')

现在CPU创造网络，再在GPU进行计算

In [ ]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 将CPU上的数据挪到某个GPU上

net(X)